In [16]:
#%%writefile get_bookprice.py
#%load get_bookprice.py
#%run get_bookprice.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
from django.db.models import Q
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq
#
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
#
import requests
import urllib.parse
import pandas as pd
import numpy as np
import random
import re
import json
import csv
#
from get_proxy import get_proxy
from get_mollie import get_mollie
#
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'wtb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from mainsite.models import Bookinfo,Bookprice,Store
from dict_stores import url_qs

In [24]:
isbn='9789863125501'
isbn13=''
#露天_https://rtapi.ruten.com.tw/api/search/v2/index.php/m/core/prod?q=9789571371870
url_q='http://www.mollie.com.tw/Mobile/Books.asp'#+isbn+"+"+isbn13
#print(url_q)
#
fake_header = Headers(
    browser="chrome",  # Generate only Chrome UA
    os="win",  # Generate ony Windows platform
    headers=True  # generate misc headers
)    
UA=fake_header.generate()  
ippo=get_proxy(which="OK",now=True)
proxies={
        "http": "http://"+ippo,
        #"https": "http://"+ippo
        } 
r = requests.post(url_q, 
                 data={'BarCode':'9789570849707'},
                 headers=UA,
                 #proxies=proxies,
                 cookies={'CheckID':'CK20200321113332'},
                 #allow_redirects=False,
                 timeout=20)    
r.encoding='big5'
print(r.text)
#
doc=pq(r.text)
r.close()


<!doctype html>
<html><!-- #BeginTemplate "/Templates/Mobile.dwt" --><!-- DW6 -->
<head>
<meta charset="big5">
<title>茉莉二手書店</title>
<meta http-equiv="Content-Type" content="text/html; charset=big5" />
<meta name="author" content="茉莉二手書店">
<meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=0">
<meta name="apple-touch-fullscreen" content="yes">
<meta name="apple-mobile-web-app-capable" content="yes">
<meta name="apple-mobile-web-app-status-bar-style" content="black">
<meta name="format-detection" content="telephone=no">
<script type="text/javascript" src="js/jquery.min.js"></script>
<script type="text/javascript" src="js/script.js"></script>
<link href="style/style.css" rel="stylesheet" type="text/css">
<style type="text/css">
.active {display:none;}
#menu_block {padding-top:60px;}
</style>
</head>
<body>
<div class="heander">
<span class="h_menu" id="sys_menu" ><a href="#"><img src="images/h_menu.png"></a></span>
<div c

In [20]:
doc.find('section#A').size()

0

In [ ]:
price_sale=doc.find('.pricing .price').text().replace('售價: $','') or ''
url_book=doc.find('.cover').attr('href') or ''
if url_book:
    url_book='https://www.tenlong.com.tw'+url_book    

price_sale,url_book

In [ ]:
[1,2,3,]